## Playground Description
This notebook will probably always be a bit 'messy' as it is a place I use to troubleshoot and test new ideas that are still being refined. 
<br>
<br>
Once a concept has been tested to an acceptable point, I will try to transfer it to the 'component-lab' notebook which will be a semi-final testing ground for what I believe is ready to be implemented

In [2]:
import json
import requests
from openai import OpenAI

In [ ]:
response = requests.post(
    'http://localhost:11434/api/generate',
    json = {
        'model': 'qwen3:8b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

response.raise_for_status()

Run time for the 8b model was 3 min 40 sec

In [ ]:
data = response.json()

In [ ]:
data["response"]

In [ ]:
data

In [ ]:
response = requests.post(
    'http://localhost:11434/api/generate',
    json = {
        'model': 'qwen3:14b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

response.raise_for_status()

trying to run the 14b model took 9 min 16 sec, perhaps I need to check the GPU driver on my laptop?

Curious to see what the difference may be if I offload this to Coruscant?

In [ ]:
data = response.json()

In [ ]:
data['response']

In [ ]:
response = requests.post(
    'http://coruscant.local:11434/api/generate',
    json = {
        'model': 'qwen3:14b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

Well ok then, running on Coruscant returned 3x faster for the 14b model...
Second run is about just as fast 3 min 48 seconds

In [ ]:
data = response.json()

In [ ]:
print(data['response'])

The issue I was having with previous output was that the string needed to be printed by python first. By extracting it "raw" it was just a representation of the string with added characters. 

Curious to see how fast the 8b model performs on Coruscant

In [ ]:
response = requests.post(
    'http://coruscant.local:11434/api/generate',
    json = {
        'model': 'qwen3:8b',
        'prompt': 'Can you explain Python Lambda functions?',
        'stream': False,
        'options': {
            'num_ctx': 20000
        }
    }
)

First run... 32 seconds!

In [ ]:
data = response.json()

In [ ]:
print(data['response'])

In [ ]:
def get_response(prompt: str, model: str = 'qwen3:8b', ctx: int = 10000) -> str:
    response = requests.post(
    'http://coruscant.local:11434/api/generate',
        json = {
            'model': model,
            'prompt': prompt,
            'stream': False,
            'options': {
                'num_ctx': ctx
            }
        }
    )

    data = response.json()

    if 'response' not in data:
        print('Unexpected output:', data)
        return ""
    
    return data['response']

From what I have seen it is possible to utilize the OpenAI method/syntax while still targeting ollama

In [2]:
base_url = 'http://coruscant.local:11434/v1'

ollama = OpenAI(base_url=base_url, api_key='ollama')

In [3]:
response = ollama.chat.completions.create(model= 'qwen3:8b', messages= [{'role': 'user', 'content': 'What are decorators in Python?'}])

response.choices[0].message.content

'Decorators in Python are a powerful feature that allows you to modify or enhance functions, methods, or classes without altering their original code. They act as **wrappers** that add functionality to existing code, promoting modularity and separation of concerns. Here\'s a structured explanation:\n\n---\n\n### **Key Concepts**\n1. **Decorator Syntax**:\n   - The `@` symbol is used to apply a decorator to a function or class.\n   - Example:\n     ```python\n     @decorator_function\n     def my_function():\n         pass\n     ```\n   - This is equivalent to:\n     ```python\n     def my_function():\n         pass\n     my_function = decorator_function(my_function)\n     ```\n\n2. **Decorator Structure**:\n   - A decorator is typically a **function or class** that takes the target function as an argument and returns a **new function** that wraps the original.\n   - Example:\n     ```python\n     def my_decorator(func):\n         def wrapper():\n             print("Before function call

In [4]:
print(response.choices[0].message.content)

Decorators in Python are a powerful feature that allows you to modify or enhance functions, methods, or classes without altering their original code. They act as **wrappers** that add functionality to existing code, promoting modularity and separation of concerns. Here's a structured explanation:

---

### **Key Concepts**
1. **Decorator Syntax**:
   - The `@` symbol is used to apply a decorator to a function or class.
   - Example:
     ```python
     @decorator_function
     def my_function():
         pass
     ```
   - This is equivalent to:
     ```python
     def my_function():
         pass
     my_function = decorator_function(my_function)
     ```

2. **Decorator Structure**:
   - A decorator is typically a **function or class** that takes the target function as an argument and returns a **new function** that wraps the original.
   - Example:
     ```python
     def my_decorator(func):
         def wrapper():
             print("Before function call")
             func()
     

Working on the Wikipedia Function

In [37]:
import wikipediaapi

In [38]:
wiki_obj = wikipediaapi.Wikipedia(user_agent='TopicSearch (grant.mlanier@gmail.com)', language='en')

In [39]:
wave = wiki_obj.page('The Great Wave off Kanagawa')

In [30]:
wave.exists()

True

In [31]:
print(wave.summary)

The Great Wave off Kanagawa (Japanese: 神奈川沖浪裏, Hepburn: Kanagawa-oki Nami Ura; lit. 'Under the Wave off Kanagawa') is a woodblock print by the Japanese ukiyo-e artist Hokusai (1760–1849), created in late 1831 during the Edo period of Japanese history. The print depicts three boats moving through a storm-tossed sea, with a large, cresting wave forming a spiral in the centre over the boats and Mount Fuji in the background.
The print is Hokusai's best-known work and the first in his series Thirty-six Views of Mount Fuji, in which the use of Prussian blue revolutionised Japanese prints. The composition of The Great Wave is a synthesis of traditional Japanese prints and use of graphical perspective developed in Europe, and earned him immediate success in Japan and later in Europe, where Hokusai's art inspired works by the Impressionists. Several museums throughout the world hold copies of The Great Wave, many of which came from 19th-century private collections of Japanese prints. Only about

In [32]:
def get_wiki(topic: str) -> str:
    wiki_obj = wikipediaapi.Wikipedia(
        user_agent='TopicSearch (grant.mlanier@gmail.com)',
        language='en'
    )
    page = wiki_obj.page(topic)

    return (
        page.summary
        if page.exists()
        else f"No page found on Wikipedia for {topic}"
    )


In [33]:
print(get_wiki('The Great Wave off Kanagawa'))

The Great Wave off Kanagawa (Japanese: 神奈川沖浪裏, Hepburn: Kanagawa-oki Nami Ura; lit. 'Under the Wave off Kanagawa') is a woodblock print by the Japanese ukiyo-e artist Hokusai (1760–1849), created in late 1831 during the Edo period of Japanese history. The print depicts three boats moving through a storm-tossed sea, with a large, cresting wave forming a spiral in the centre over the boats and Mount Fuji in the background.
The print is Hokusai's best-known work and the first in his series Thirty-six Views of Mount Fuji, in which the use of Prussian blue revolutionised Japanese prints. The composition of The Great Wave is a synthesis of traditional Japanese prints and use of graphical perspective developed in Europe, and earned him immediate success in Japan and later in Europe, where Hokusai's art inspired works by the Impressionists. Several museums throughout the world hold copies of The Great Wave, many of which came from 19th-century private collections of Japanese prints. Only about

In [34]:
print(get_wiki('asdgf'))

No page found on Wikipedia for asdgf


Trying to create tools to use for the LLM to use. Below is a sample of the syntax with the first example of searching Wikipedia.

In [54]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_wiki",
            "description": "Fetch the summary for a topic from Wikipedia (or return a not-found message).",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic to search Wikipedia for"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    }
]


In [55]:
base_url = 'http://coruscant.local:11434/v1'

ollama = OpenAI(base_url=base_url, api_key='ollama')

In [67]:
msg = [
    {
        'role': 'user',
        'content': 'what can you tell me about The Great Wave off Kanagawa?'
    }
] 

In [ ]:
response = ollama.chat.completions.create(
    model='qwen3:8b',
    tools=tools,
    messages= msg
)

print(response.choices[0].message.content)

In [57]:
response.dict()

C:\Users\grant\AppData\Local\Temp\ipykernel_21644\3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  response.dict()


{'id': 'chatcmpl-292',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': '',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_e0m04a8z',
      'function': {'arguments': '{"topic":"The Great Wave off Kanagawa"}',
       'name': 'get_wiki'},
      'type': 'function',
      'index': 0}],
    'reasoning': 'Okay, the user is asking about "The Great Wave off Kanagawa." I remember that\'s a famous woodblock print. Let me think. It\'s by Hokusai, right? The title might be "Fujiwara no Michikatsu" or something similar. Wait, no, the correct title is "The Great Wave off Kanagawa" (Kanagawa Okino Fujō). The artwork depicts a massive wave with boats in the foreground and Mount Fuji in the background. It\'s part of the Thirty-Six Views of Mount Fuji series. Hokusai created this in the Edo period. The wave is actually a tsunami, and the composit

In [66]:
msg = [
    {
        'role': 'user',
        'content': 'what can you tell me about The Great Wave off Kanagawa?'
    }
]

In [ ]:
response = ollama.chat.completions.create(
    model='qwen3:8b',
    messages= msg    
)

print(response.choices[0].message.content)

**"The Great Wave off Kanagawa"** (Kanagawa Okinawamura / Kanagawa Okinawa-maru) is one of the most iconic and celebrated woodblock prints in the world. Here's a detailed overview:

---

### **Artist and Context**
- **Created by**: **Katsushika Hokusai** (1760–1849), a master of **ukiyo-e** (Japanese woodblock print art), known for his intricate compositions and innovative use of perspective.
- **Series**: Part of **"Thirty-Six Views of Mount Fuji"** (1831–1832), a collection of 46 prints celebrating the Japanese mountain. The wave is the fourth print in the series.
- **Time Period**: Created during Japan's **Edo period** (1603–1868), a time of relative peace and cultural flourishing.

---

### **Artistic Description**
- **Composition**: The print features a **dramatic, curling wave** (a stylized representation of a tsunami or storm) that looms over three small fishing boats. The wave's crest is rendered with **bold, swirling lines**, creating a sense of motion and power.
- **Foregroun

In [49]:
response.dict()

C:\Users\grant\AppData\Local\Temp\ipykernel_21644\3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  response.dict()


{'id': 'chatcmpl-664',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '**"The Great Wave off Kanagawa"** (Kanagawa Okinawamura / Kanagawa Okinawa-maru) is one of the most iconic and celebrated woodblock prints in the world. Here\'s a detailed overview:\n\n---\n\n### **Artist and Context**\n- **Created by**: **Katsushika Hokusai** (1760–1849), a master of **ukiyo-e** (Japanese woodblock print art), known for his intricate compositions and innovative use of perspective.\n- **Series**: Part of **"Thirty-Six Views of Mount Fuji"** (1831–1832), a collection of 46 prints celebrating the Japanese mountain. The wave is the fourth print in the series.\n- **Time Period**: Created during Japan\'s **Edo period** (1603–1868), a time of relative peace and cultural flourishing.\n\n---\n\n### **Artistic Description**\n- **Composition**: The print features a **dramatic, curling wave** (a stylized representation of a tsunami or storm) that looms over

It looks like I was able to pass the tool to the model but it did not use it. Assuming this is becuase it was able to reason enough within its training data. I would need to find a way to prompt the LLM for something that it would have to pull from Wikipedia.

In [65]:
response.choices[0].message.tool_calls[0].function.arguments

'{"topic":"The Great Wave off Kanagawa"}'

In [68]:
for tool_call in response.choices[0].message.tool_calls:
    if tool_call.function.name == 'get_wiki':
        args = json.loads(tool_call.function.arguments)
        topic = get_wiki(args['topic'])

        msg.append(
            {
                'role': 'tool',
                'tool_call_id': tool_call.id,
                'content': json.dumps({'topic': topic})
            }
        )

response = ollama.chat.completions.create(
    model='qwen3:8b',
    messages=msg,
    tools=tools
)

print(response.choices[0].message.content)

"The Great Wave off Kanagawa" (Japanese: *Kanagawa-oki Nami Ura*) is a iconic woodblock print by the 19th-century Japanese artist Katsushika Hokusai (1760–1849), created in 1831 during Japan’s Edo period. It is the first print in Hokusai’s series *Thirty-six Views of Mount Fuji* and showcases his mastery of ukiyo-e (woodblock printing).  

The composition features a towering, curling wave—symbolizing the raw power of nature—threatening three boats adrift in a stormy sea. In the background, the serene silhouette of Mount Fuji contrasts with the chaotic waves, embodying the harmony between natural elements. Hokusai’s use of **Prussian blue** (a newly introduced pigment) and European-inspired linear perspective revolutionized Japanese printmaking, blending traditional Japanese aesthetics with global artistic trends.  

This print became a cultural phenomenon, inspiring Western artists like **Vincent van Gogh**, **Claude Monet**, and **Claude Debussy**, who referenced it in their works. To

In [83]:
today = wiki_obj.page('january_24')

today_secs = today.sections
# type(today_secs[0])
today_secs[0].section_by_title('Pre-1600')


Section: Pre-1600 (2):
41 – Claudius is proclaimed Roman emperor by the Praetorian Guard after they assassinate the previous emperor, his nephew Caligula.
914 – Start of the First Fatimid invasion of Egypt.
1438 – The Council of Basel suspends Pope Eugene IV.
1458 – Matthias Corvinus is elected King of Hungary.
1536 – King Henry VIII of England suffers an accident while jousting, leading to a brain injury that historians say may have influenced his later erratic behaviour and possible impotence.
Subsections (0):

In [97]:
for sec in today_secs:
    print(sec.title)

Events
Births
Deaths
Holidays and observances
References
External links


In [103]:
for sec in today_secs[0].sections:
    print(sec.text + '\n\n')

41 – Claudius is proclaimed Roman emperor by the Praetorian Guard after they assassinate the previous emperor, his nephew Caligula.
914 – Start of the First Fatimid invasion of Egypt.
1438 – The Council of Basel suspends Pope Eugene IV.
1458 – Matthias Corvinus is elected King of Hungary.
1536 – King Henry VIII of England suffers an accident while jousting, leading to a brain injury that historians say may have influenced his later erratic behaviour and possible impotence.


1651 – Arauco War: Spanish and Mapuche authorities meet in the Parliament of Boroa renewing the fragile peace established at the parliaments of Quillín in 1641 and 1647.
1679 – King Charles II of England dissolves the Cavalier Parliament.
1742 – Charles VII Albert becomes Holy Roman Emperor.
1758 – During the Seven Years' War the leading burghers of Königsberg submit to Elizabeth of Russia, thus forming Russian Prussia (until 1763).
1817 – Crossing of the Andes: Many soldiers of Juan Gregorio de las Heras are captu

In [107]:
today_secs[0].sections[0].text.split('\n')

['41 – Claudius is proclaimed Roman emperor by the Praetorian Guard after they assassinate the previous emperor, his nephew Caligula.',
 '914 – Start of the First Fatimid invasion of Egypt.',
 '1438 – The Council of Basel suspends Pope Eugene IV.',
 '1458 – Matthias Corvinus is elected King of Hungary.',
 '1536 – King Henry VIII of England suffers an accident while jousting, leading to a brain injury that historians say may have influenced his later erratic behaviour and possible impotence.']

In [128]:
wiki_obj = wikipediaapi.Wikipedia(user_agent='TopicSearch (grant.mlanier@gmail.com)', language='en')
cur_event = wiki_obj.page('Portal:Current events')

# cur_event.exists()
print(cur_event.text)
cur_event.fullurl

'https://en.wikipedia.org/wiki/Portal:Current_events'

In [129]:
len(cur_event.summary)

0

In [130]:
[s.title for s in cur_event.sections]

[]

In [131]:
print(cur_event.exists())
print(cur_event.fullurl)
print("summary len:", len(cur_event.summary or ""))
print("text len:", len(cur_event.text or ""))
print("sections:", [s.title for s in cur_event.sections][:10])


True
https://en.wikipedia.org/wiki/Portal:Current_events
summary len: 0
text len: 0
sections: []


It appears that the 'Current Events' page does not return a value using the wikipedia api because of some issue with templates/transclusions(?) The suggestion is to use a direct call to the MediaWiki API using requests and bs4

### Trying to build out more organized 'class' structure / algin more with OOP principals  
Going through some of the udemy course, most start off using just a simple 'function' approach while teaching, but (in at least one of them) switches to using 'class' or OOP focused methods. This, based on my research, should help reduce some of the repetitive nature of things like making the call to the the LLM client. This will be the first class that I work on with the idea that any agent that follows would end up relying on it. 

The following is the product of working through some of the above code with GPT.

In [49]:
from openai import OpenAI
from typing import Iterable, Optional
from IPython.display import Markdown, display # for displaying formatted text in Jupyter
import json # added after moving the tool agent to be 'chat focused'
import requests # added to test simple tool calls
from dotenv import load_dotenv # added to protect API keys
import os
import wikipediaapi # needed for the get_wiki tool

load_dotenv(override=True)
open_weather_api_key = os.getenv('OPEN_WEATHER_API_KEY')

In [6]:
class CallLlmClient:
    def __init__(
        self,
        base_url: str,
        model: str,
        api_key: str = "ollama",
        default_tools: Optional[list] = None,
    ):
        self.model = model
        self.default_tools = default_tools or []

        self.client = OpenAI(
            base_url=base_url,
            api_key=api_key,
        )

    def chat(
        self,
        messages: list[dict],
        tools: Optional[list] = None,
        **kwargs,
    ) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=tools or self.default_tools,
            **kwargs,
        )

        return response.choices[0].message.content


next this is a simple example of its usage

In [10]:
msg = [
    {
        'role': 'user',
        'content': 'Who was the artist of The Great Wave off Kanagawa?'
    }
]

llm = CallLlmClient(
    base_url="http://localhost:11434/v1",
    model="qwen3:8b",
)

reply = llm.chat(msg)
display(Markdown(reply))


The artist of *The Great Wave off Kanagawa* (also known as *The Great Wave* or *Kanagawa Kani* in Japanese) is **Katsushika Hokusai** (1769–1849). This iconic woodblock print is part of his series *Thirty-Six Views of Mount Fuji*, created in the 1830s. Hokusai was a renowned Japanese ukiyo-e (woodblock print) artist from the Edo period, and this work remains one of the most celebrated and recognizable images in world art.

As I was working through some questions about building agents, GPT called out the above LLM class is not really setup to handle tool calls (despite it taking it as a parameter in the init method??) 
<br>
<br>
From what I can see the change it recommends would return the raw 'message' object instead of 'message.content' like the original does. This is needed to determine if the first call/message returned a tool call it seems so in the case that it does it can then be leveraged instead of just returning the initial output. 

In [2]:
# Edited class that allows Agent to determine what to do with message
class LlmClient:
    def __init__(self, base_url: str, model: str, api_key: str = "ollama"):
        self.model = model
        self.client = OpenAI(base_url=base_url, api_key=api_key)

    def chat(
        self,
        messages: list[dict],
        tools: Optional[list] = None,
        **kwargs,
    ):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=tools,
            **kwargs,
        )

        return response.choices[0].message

now with this class we also set up the Base Agent class to make sure the agents return the output. Then specialized agent classes would be made to handle the message differently depending on the situation and then manipulate things further before output.

In [13]:
class BaseAgent:
    def __init__(self, llm: LlmClient, system_prompt: str):
        self.llm = llm
        self.system_prompt = system_prompt

    def build_messages(self, user_input: str) -> list[dict]:
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_input},
        ]

    def run(self, user_input: str) -> str:
        message = self.llm.chat(self.build_messages(user_input))
        return message.content


This next class is a demo of a class GPT made that would utilize the tools if they are relevant and called upon it seems. 

In [14]:
class ToolAgent(BaseAgent):
    def __init__(self, llm: LlmClient, tools: list):
        super().__init__(
            llm,
            system_prompt="You may call tools when appropriate.",
        )
        self.tools = tools

    def run(self, user_input: str):
        message = self.llm.chat(
            messages=self.build_messages(user_input),
            tools=self.tools,
        )

        # Model wants to call a tool
        if message.tool_calls:
            return message  # or dispatch tool execution

        return message.content


Looking over things though, it looks like this ToolAgent class is not actually set up to handle the tools (which I guess makes sense seeing as I have not exposed any to it yet) and GPT even calls out that it can help build a 'execution loop' which would allow the tool agent to be utilized it seems. 

This following class is redesigned to handle a message history and be more 'chat' focused as well. Speaking with GPT about the history, it recommends keeping the history's 'state' within the agent itself instead of relating it back to the LLMClient. This way new agents will not be impacted by previous agents messages that may cause issues. (like additional tool calls?)
> I am curious though as to how (if needed) I would be able to break off a section of the history to pass to a different agent if needed. It is a structured object so there should be some way I would think but may need to get creative in how I pull the index or value. 

In [39]:
class ChatToolAgent:
    def __init__(
        self,
        llm: "LlmClient",
        tools: list,
        tool_dispatch: dict,
        system_prompt: str = "You are a helpful assistant. Use tools when helpful.",
        max_steps: int = 8,
        max_history_msgs: Optional[int] = 50,
    ):
        self.llm = llm
        self.tools = tools
        self.tool_dispatch = tool_dispatch
        self.system_prompt = system_prompt
        self.max_steps = max_steps
        self.max_history_msgs = max_history_msgs

        # Conversation state lives here
        self.messages: list[dict] = []
        self.reset()

    def reset(self):
        """Start a fresh conversation."""
        self.messages = [{"role": "system", "content": self.system_prompt}]

    def _trim_history(self):
        """
        Keep system + last N messages.
        (This is a simple heuristic; later you can summarize older turns.)
        """
        if self.max_history_msgs is None:
            return
        # always keep the system message at index 0
        if len(self.messages) > (1 + self.max_history_msgs):
            self.messages = [self.messages[0]] + self.messages[-self.max_history_msgs :]

    def ask(self, user_input: str) -> str:
        """
        One user turn, but may require multiple internal steps if tools are called.
        Conversation history is preserved across calls.
        """
        # Add user message to ongoing history
        self.messages.append({"role": "user", "content": user_input})
        self._trim_history()

        for _ in range(self.max_steps):
            # 1) model call
            message = self.llm.chat(messages=self.messages, tools=self.tools)

            # append assistant message to history
            self.messages.append(message.model_dump())
            self._trim_history()

            # 2) if no tool calls -> return assistant content
            if not message.tool_calls:
                return message.content or ""

            # 3) execute each requested tool call
            for call in message.tool_calls:
                tool_name = call.function.name
                raw_args = call.function.arguments or "{}"

                try:
                    args = json.loads(raw_args)
                except json.JSONDecodeError:
                    result = f"ERROR: Invalid JSON arguments: {raw_args}"
                else:
                    func = self.tool_dispatch.get(tool_name)
                    if not func:
                        result = f"ERROR: Unknown tool '{tool_name}'"
                    else:
                        try:
                            result = func(**args)
                        except Exception as e:
                            result = f"ERROR: Tool '{tool_name}' failed: {e}"

                # 4) feed tool result back
                self.messages.append(
                    {
                        "role": "tool",
                        "tool_call_id": call.id,
                        "name": tool_name,
                        "content": str(result),
                    }
                )
                self._trim_history()

        return "ERROR: Reached max tool steps without a final answer."


In [3]:
msg = [
    {
        'role': 'user',
        'content': 'Hello, How are you?'
    }
]

llm = LlmClient(
    base_url="http://localhost:11434/v1",
    model="qwen3:8b",
)

In [4]:
response = llm.chat(msg)

In [7]:
# print(response)
response.model_dump()

{'content': "Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help! How are you today? 😊",
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None,
 'reasoning': 'Okay, the user asked "Hello, How are you?" I need to respond appropriately. Since I\'m an AI, I don\'t have feelings, so I should mention that I don\'t have emotions. But I should also be friendly and ask how they\'re doing. Let me check if there\'s any specific context I should consider. The user might just be greeting me, so a simple, cheerful response should work. I\'ll make sure to keep it conversational and open-ended in case they have more to say.\n'}

In [10]:
response.content

"Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help! How are you today? 😊"

In [11]:
type(response)

openai.types.chat.chat_completion_message.ChatCompletionMessage

In [12]:
type(msg)

list

In [13]:
type(msg[0])

dict

In [15]:
msg[0].values()

dict_values(['user', 'Hello, How are you?'])

In [16]:
msg = [
    {
        'role': 'user',
        'content': 'Can you tell me the weather in Hammond Louisiana?'
    }
]
response = llm.chat(msg)
display(Markdown(response.content))

I don't have access to real-time weather data, so I can't provide current conditions for Hammond, Louisiana. However, here's a general overview of the typical weather in the area:

### **Typical Weather in Hammond, Louisiana:**
- **Summer (June–August):**  
  - **Highs:** 90°F (32°C) or warmer.  
  - **Lows:** 75°F (24°C) or higher.  
  - **Conditions:** Humid, with frequent thunderstorms and heavy rainfall, especially in July and August.  

- **Winter (December–February):**  
  - **Highs:** 65°F (18°C) to 75°F (24°C).  
  - **Lows:** 50°F (10°C) to 55°F (13°C).  
  - **Conditions:** Mild, with occasional rain or light snow.  

- **Spring (March–May) & Fall (September–November):**  
  - **Highs:** 75°F (24°C) to 85°F (29°C).  
  - **Lows:** 60°F (16°C) to 70°F (21°C).  
  - **Conditions:** Variable, with warm days and cooler evenings. Spring often brings thunderstorms, while fall can have cooler temperatures and occasional rain.  

### **Tips for Real-Time Updates:**  
For current conditions, check services like:  
- **National Weather Service (NWS):** [www.weather.gov](https://www.weather.gov)  
- **AccuWeather** or **The Weather Channel** apps.  
- **Local news stations** (e.g., KALB, KPLC, or KATC).  

Let me know if you’d like help finding a specific resource! 🌤️

The code in the cell above shows that the default model does not have access to real time data (sanity check) Below I used GPT to write up a quick weather checking function, to test out the tool agent.

In [29]:
url = "https://api.openweathermap.org/data/2.5/weather"
api_key = open_weather_api_key
location = "Hammond,Louisiana,US"
params = {
        "q": location,
        "appid": api_key,
        "units": "imperial"  # Fahrenheit
    }

response = requests.get(url, params=params)

In [30]:
response.json()

{'coord': {'lon': -90.4612, 'lat': 30.5044},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 57.2,
  'feels_like': 55.35,
  'temp_min': 57.2,
  'temp_max': 57.2,
  'pressure': 1018,
  'humidity': 58,
  'sea_level': 1018,
  'grnd_level': 1018},
 'visibility': 10000,
 'wind': {'speed': 10.36, 'deg': 350},
 'clouds': {'all': 75},
 'dt': 1769804506,
 'sys': {'type': 1,
  'id': 4342,
  'country': 'US',
  'sunrise': 1769777644,
  'sunset': 1769816164},
 'timezone': -21600,
 'id': 4326868,
 'name': 'Hammond',
 'cod': 200}

Ran into an issue where open weather's api was not showing the key as valid, says it can take up to 2 hours to show up.. trying something else..
> coming back to it the API is working now so looks like I have two methods to pull weather data now. :)

In [31]:
data = response.json()

In [32]:
data['main']

{'temp': 57.2,
 'feels_like': 55.35,
 'temp_min': 57.2,
 'temp_max': 57.2,
 'pressure': 1018,
 'humidity': 58,
 'sea_level': 1018,
 'grnd_level': 1018}

In [33]:
data['main']['temp']

57.2

In [34]:
def get_weather(city: str, state: str = None, api_key:str = open_weather_api_key) -> dict:
    if not api_key:
        return {"error": "No API key provided."}
    
    location = f"{city},{state},US" if state else f"{city},US"
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = { 
        "q": location,
        "appid": api_key,
        "units": "imperial"  # Fahrenheit
    }
   
    response = requests.get(url, params=params)
    response.raise_for_status()

    data = response.json()

    return {
        "city": data["name"],
        "temperature_f": data["main"]["temp"],
        "feels_like_f": data["main"]["feels_like"],
        "humidity": data["main"]["humidity"],
        "conditions": data["weather"][0]["description"],
        "wind_mph": data["wind"]["speed"]
    }


Moving the `get_wiki` function down for ease fo viewing

In [35]:
def get_wiki(topic: str) -> str:
    wiki_obj = wikipediaapi.Wikipedia(
        user_agent='TopicSearch (grant.mlanier@gmail.com)',
        language='en'
    )
    page = wiki_obj.page(topic)

    return (
        page.summary
        if page.exists()
        else f"No page found on Wikipedia for {topic}"
    )


now we create the tools list. I noticed that the two udemy courses use slightly different methods. Where the first one did it all in one go, the second formats the inner function call first then passes that to a smaller format of the overall. The second method looks like it could be used to help keep things compact but will look into that later. 

In [36]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_wiki",
            "description": "Fetch the summary for a topic from Wikipedia (or return a not-found message).",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic to search Wikipedia for"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a specified city (and optional state) in the US.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather for"
                    },
                    "state": {
                        "type": "string",
                        "description": "The state abbreviation (optional)"
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    }
]


In [37]:
tools_dispatch = {
    "get_wiki": get_wiki,
    "get_weather": get_weather,
}

In [40]:
gen_agent = ChatToolAgent(llm, tools, tools_dispatch)

In [42]:
response = gen_agent.ask('Can you tell me the weather in Hammond Louisiana?')
display(Markdown(response))

The current weather in Hammond, LA remains:  
- **Temperature**: 59°F (feels like 56.66°F)  
- **Humidity**: 44%  
- **Conditions**: Broken clouds  
- **Wind**: 10.36 mph  

Let me know if you'd like updates or additional details! 😊

After a bit of back and forth to confirm the process I would want to use to call the functions, it looks like things are working well with the weather tool but I am curious to see if the wiki tool makes an actual impact. Below is a 'raw' call to the llm asking it about the town I live in, I will then pass that to the tool agent to compare if it was able to figure if it should check wiki or not.

In [45]:
msg = [
    {
        'role': 'user',
        'content': 'What can you tell me about Hammond Louisiana?'
    }
]
response = llm.chat(msg)
display(Markdown(response.content))

Hammond, Louisiana, is a city located in **St. Tammany Parish**, part of the **greater New Orleans metropolitan area** (approximately 35 miles northwest of New Orleans). Here’s an overview of the city:

---

### **Key Details**
- **Population**: Approximately **28,000** (as of recent estimates).
- **Founded**: Incorporated in the **1950s** as a planned suburban community.
- **Location**: Situated along the **Mississippi River** and near the **Gulf Coast**, with access to the **Bayou La Batre** area.

---

### **Economy & Geography**
- **Economic Base**: Primarily residential and commercial development, including retail, healthcare, and light industry.
- **Geography**: The city sits in a flood-prone region due to its low-lying terrain and proximity to the Mississippi River. It is part of the **New Orleans Flood Protection System**, which includes levees and floodwalls.
- **Proximity**: Adjacent to **Slidell**, **Mandeville**, and **Metairie**, making it a suburban hub with easy access to New Orleans.

---

### **Culture & Community**
- **Recreational Areas**: 
  - **St. Tammany Parish Park** (offers boating, fishing, and camping).
  - **Bayou La Batre** (a popular freshwater lake for recreation).
  - **Hammocks Creek** and **Pine Lake** for outdoor activities.
- **Cultural Attractions**: 
  - The **Hammond Historical Society** and local museums.
  - Annual events like the **Hammond Harvest Festival** (celebrating agriculture and community).
- **Local Flavor**: Known for its blend of Southern hospitality, Creole influence, and family-friendly atmosphere.

---

### **Challenges**
- **Flood Risk**: Vulnerable to hurricanes, heavy rainfall, and river flooding (e.g., during Hurricane Katrina in 2005).
- **Environmental Issues**: Ongoing concerns about coastal erosion and water management in the region.

---

### **Notable Fact**
- **Hammond, Louisiana**, is distinct from **Hammond, Indiana**, and the **Hammond, Texas** area. The Louisiana city is part of the **New Orleans metropolitan statistical area**, while others are in different states.

---

If you’re looking for specific details (e.g., schools, businesses, or historical landmarks), feel free to ask!

It says Hammond is in St Landry Parish???? That is clearly wrong, but I wonder if that is just this attempt, and future attempts would be better? (run time is around 25-30 seconds)
> - Second attempt said it was in Ascension Parish.....
> - At least this confirms that future attempts would be different but how often does it get it right? vs will the tool call be more accurate.
> - third call said st tammany....
Lets try the tool call

In [47]:
response = gen_agent.ask('What can you tell me about Hammond Louisiana?')
display(Markdown(response))

It seems there's a technical limitation in accessing external resources like Wikipedia. However, here's a summary of **Hammond, Louisiana** based on available knowledge:

### **Hammond, Louisiana**  
- **Location**: A city in Jefferson Parish, approximately 10 miles north of New Orleans.  
- **History**:  
  - Founded in 1836, it was a key stop on the Natchez Trace Parkway.  
  - Known as the "Cradle of the Blues" for its jazz and music heritage.  
  - Played a role in the Civil War, including the Battle of Hammond (1862).  
- **Notable Landmarks**:  
  - **Jefferson Parish Courthouse** (built in 1852, listed on the National Register of Historic Places).  
  - **Hammond Park** (a historic district with antebellum homes).  
  - **St. Louis Cathedral** (part of the New Orleans metropolitan area).  
- **Culture**: Rich in Creole and French heritage, with vibrant music scene and proximity to cultural hubs like New Orleans.  
- **Population**: ~50,000 (as of recent estimates).  

Let me know if you'd like details about specific landmarks, history, or local culture! 😊

So it looks like it failed but the cool thing is it seems like the llm was able to tell there was in an issue in both attempts that I have ran so far. GPT did mention something about pulling out the history, maybe implementing this will help me see what is going on
> may want to double check the get_wiki call too. 

In [50]:
get_wiki('Hammond Louisiana')

'No page found on Wikipedia for Hammond Louisiana'

looks like I may just have forgotten to import the library....
<br>
<br>
even after importing it There is still an error finding the page it seems. 

In [56]:
wiki_obj = wikipediaapi.Wikipedia(
    user_agent='TopicSearch (grant.mlanier@gmail.com)',
    language='en'
)
page = wiki_obj.page('Hammond, Louisiana')
page.exists()

True

So it looks like there may need to be some formatting that would have needed to be done? I found that if I left the comma ',' from the City, State format the wiki page will not be found. Wonder if that is the case about other cities?

In [57]:
response = gen_agent.ask('What can you tell me about Hammond, Louisiana?')
display(Markdown(response))

Hammond, Louisiana, is a city in Jefferson Parish, located just north of New Orleans. Here’s a concise overview:  

### **Key Highlights**  
- **History**:  
  - Founded in 1836, it was a stop on the Natchez Trace Parkway and played a role in the Civil War (e.g., the 1862 Battle of Hammond).  
  - Known as the "Cradle of the Blues" for its jazz heritage and cultural influence.  

- **Landmarks**:  
  - **Jefferson Parish Courthouse** (1852, historic architecture).  
  - **Hammond Park** (a National Historic Landmark with antebellum homes).  
  - **St. Louis Cathedral** (part of the New Orleans metropolitan area).  

- **Culture & Economy**:  
  - A hub for Creole and French heritage, with vibrant music and cuisine.  
  - Proximity to New Orleans offers easy access to cultural attractions.  

- **Demographics**:  
  - Population ~50,000 (as of recent estimates).  

If you’d like deeper insights into specific aspects (e.g., history, architecture, or local events), feel free to ask! 😊

Still got the parish wrong... maybe it is in the way I am asking the question? Perhaps I should ask it specifically about the parish since it seems to be getting that wrong more often than not. 

In [58]:
response = gen_agent.ask('What parish is Hammond, Louisiana in?')
display(Markdown(response))

Hammond, Louisiana, is located in **Jefferson Parish**, which is just north of New Orleans. The parish is part of the New Orleans metropolitan area and is known for its historical and cultural landmarks. 😊

In [59]:
response = gen_agent.ask('Can you check wikipedia for Hammond, Louisiana?')
display(Markdown(response))

I’m unable to access Wikipedia or external tools at the moment, but I can share that **Hammond, Louisiana** is located in **Jefferson Parish**, just north of New Orleans. Let me know if you’d like details about its history, culture, or landmarks! 😊

Looks like it is still having issues making the connection to the wiki tool maybe? or perhaps it is still not passing the formatting correct? This sounds like something that could be a good opportunity for multi shot prompting?

To make sure and for future issues below is a refactor of the Tool Agent code to include the option to call a history of the tools called as well as a function to specifically call the last used tool. 

In [60]:
class ChatToolAgent:
    def __init__(
        self,
        llm: "LlmClient",
        tools: list,
        tool_dispatch: dict,
        system_prompt: str = "You are a helpful assistant. Use tools when helpful.",
        max_steps: int = 8,
        max_history_msgs: Optional[int] = 50,
    ):
        self.llm = llm
        self.tools = tools
        self.tool_dispatch = tool_dispatch
        self.system_prompt = system_prompt
        self.max_steps = max_steps
        self.max_history_msgs = max_history_msgs

        # Conversation state lives here
        self.messages: list[dict] = []
        self.reset()

    def reset(self):
        """Start a fresh conversation."""
        self.messages = [{"role": "system", "content": self.system_prompt}]

    def _trim_history(self):
        """
        Keep system + last N messages.
        (This is a simple heuristic; later you can summarize older turns.)
        """
        if self.max_history_msgs is None:
            return
        # always keep the system message at index 0
        if len(self.messages) > (1 + self.max_history_msgs):
            self.messages = [self.messages[0]] + self.messages[-self.max_history_msgs :]

    def ask(self, user_input: str) -> str:
        """
        One user turn, but may require multiple internal steps if tools are called.
        Conversation history is preserved across calls.
        """
        # Add user message to ongoing history
        self.messages.append({"role": "user", "content": user_input})
        self._trim_history()

        for _ in range(self.max_steps):
            # 1) model call
            message = self.llm.chat(messages=self.messages, tools=self.tools)

            # append assistant message to history
            self.messages.append(message.model_dump())
            self._trim_history()

            # 2) if no tool calls -> return assistant content
            if not message.tool_calls:
                return message.content or ""

            # 3) execute each requested tool call
            for call in message.tool_calls:
                tool_name = call.function.name
                raw_args = call.function.arguments or "{}"

                try:
                    args = json.loads(raw_args)
                except json.JSONDecodeError:
                    result = f"ERROR: Invalid JSON arguments: {raw_args}"
                else:
                    func = self.tool_dispatch.get(tool_name)
                    if not func:
                        result = f"ERROR: Unknown tool '{tool_name}'"
                    else:
                        try:
                            result = func(**args)
                        except Exception as e:
                            result = f"ERROR: Tool '{tool_name}' failed: {e}"

                # 4) feed tool result back
                self.messages.append(
                    {
                        "role": "tool",
                        "tool_call_id": call.id,
                        "name": tool_name,
                        "content": str(result),
                    }
                )
                self._trim_history()

        return "ERROR: Reached max tool steps without a final answer."
    
    # New additions to extract tool call info from conversation history
    def get_tool_calls(self) -> list[dict]:
        calls = []
        pending = {}

        for msg in self.messages:
            role = msg.get('role')

            if role == 'assistant' and msg.get("tool_calls"):
                for call in msg["tool_calls"]:
                    pending[call['id']] = {
                        'tool': call['function']['name'],
                        'arguments' : call['function'].get('arguments'),
                        'result': None
                    }
            elif role == 'tool':
                call_id = msg.get('tool_call_id')
                if call_id in pending:
                    pending[call_id]['result'] = msg.get('content')
                    calls.append(pending.pop(call_id))

        return calls
    
    def print_tool_calls(self) -> None:
        calls = self.get_tool_calls()
        if not calls:
            print("No tool calls made.")
            return
        for i, c in enumerate(calls, 1):
            print(f"\nTool Call #{i}")
            print(f"Tool: {c['tool']}")
            try:
                args = json.loads(c['arguments']) if c['arguments'] else {}
                print(' Arguments:')
                for k, v in args.items():
                    print (f"  - {k}: {v}")
            except Exception:
                print(f"arguments: {c['arguments']}")
            print(f" Result: \n   {c['result']}")
    
    def last_tool_call(self) -> dict | None:
        calls = self.get_tool_calls()
        return calls[-1] if calls else None

In [61]:
gen_agent = ChatToolAgent(llm, tools, tools_dispatch)

In [64]:
response = gen_agent.ask('Can you check wikipedia for Hammond, Louisiana?')
display(Markdown(response))

Hammond, Louisiana, is the largest city in Tangipahoa Parish, located 45 miles east of Baton Rouge and 45 miles northwest of New Orleans. With a population of **21,359** as of 2020, it is home to **Southeastern Louisiana University** and serves as the principal city of the Hammond Metropolitan Statistical Area. It is also part of the **Baton Rouge-Hammond Combined Statistical Area**. Let me know if you'd like further details!

Right after I add that feature it seems to work!??! was it an attempt issue again? Additional runs came back accurate as well and even faster but I think this may be due to the chat history since I was running the same cell. 

In [65]:
gen_agent.reset()

In [66]:
response = gen_agent.ask('Can you check wikipedia for Hammond, Louisiana?')
display(Markdown(response))

Hammond, Louisiana, is the largest city in Tangipahoa Parish, situated 45 miles east of Baton Rouge and 45 miles northwest of New Orleans. As of the 2020 population estimates, its population was **21,359**, up from **20,019** in the 2010 U.S. Census. 

The city is home to **Southeastern Louisiana University** and serves as the principal city of the Hammond Metropolitan Statistical Area. It is also part of the larger **Baton Rouge-Hammond Combined Statistical Area**, highlighting its regional significance.

Even after running the reset method it pulls the right info. This is a good sign I think! Not sure what the issue was previously? Maybe another history thing and when I refactored the class it basically reset it again?

In [67]:
gen_agent.print_tool_calls()


Tool Call #1
Tool: get_wiki
 Arguments:
  - topic: Hammond, Louisiana
 Result: 
   Hammond is the largest city in Tangipahoa Parish, Louisiana, United States, located 45 miles (72 km) east of Baton Rouge and 45 miles (72 km) northwest of New Orleans. Its population was 20,019 in the 2010 U.S. census, and 21,359 at the 2020 population estimates program.
Hammond is home to Southeastern Louisiana University. It is the principal city of the Hammond metropolitan statistical area, which includes all of Tangipahoa Parish and is a part of the Baton Rouge-Hammond combined statistical area.


In [68]:
page.summary

'Hammond is the largest city in Tangipahoa Parish, Louisiana, United States, located 45 miles (72 km) east of Baton Rouge and 45 miles (72 km) northwest of New Orleans. Its population was 20,019 in the 2010 U.S. census, and 21,359 at the 2020 population estimates program.\nHammond is home to Southeastern Louisiana University. It is the principal city of the Hammond metropolitan statistical area, which includes all of Tangipahoa Parish and is a part of the Baton Rouge-Hammond combined statistical area.'

In [69]:
prompt = (
    "You are a helpful assistant.\n"
    "When you call tools, you MUST use their results to produce a final answer in your own words.\n"
    "Do NOT just repeat tool output verbatim. Summarize, explain, and cite key points.\n"
    "If a tool returns long text, extract only the relevant parts."
)
new_agent = ChatToolAgent(llm, tools, tools_dispatch, system_prompt=prompt)

In [70]:
new_rep = new_agent.ask('Can you tell me what parish Hammond, Louisiana is in?')
display(Markdown(new_rep))

Hammond, Louisiana, is located in **Tangipahoa Parish**. It is the largest city in the parish and serves as its principal city. The parish is situated in southern Louisiana, approximately 45 miles east of Baton Rouge and 45 miles northwest of New Orleans.

The above prompt is recommended to help the model just not regurgitate the tools output. The final 'system prompt' will need to be tweaked eventually to include multi shot prompting as well. 